In [4]:
import pandas as pd

# Load dataset (gantilah dengan path yang sesuai)
df = pd.read_csv('Data Scientist - Assignment.csv')

df_fico_zero = df[df['FICO Score'] >= 300]
print(df_fico_zero.shape)
print(df_fico_zero)


(52955, 23)
       UniqueID  Loan To Value  Branch ID  Age Employment Type DisbursalDate  \
0        537409          73.23         67   33   Self employed    26/09/2018   
1        624493          88.48         67   25   Self employed    26/10/2018   
2        518279          89.66         67   28   Self employed    19/09/2018   
4        490213          89.56         67   27   Self employed    05/09/2018   
5        510980          86.95         67   50        Salaried    16/09/2018   
...         ...            ...        ...  ...             ...           ...   
59467    509187          66.98        254   44             NaN    15/09/2018   
59471    640581          77.97        254   32             NaN    30/10/2018   
59473    473780          65.99        254   33             NaN    29/08/2018   
59474    617679          67.21        254   31             NaN    25/10/2018   
59476    563730          75.01        254   28             NaN    08/10/2018   

       State_ID     State  